In [2]:
import json
import re

from dotenv import load_dotenv
import os

load_dotenv()


import sys
import urllib.request

import requests
from bs4 import BeautifulSoup
import pandas as pd

### 쿼리에 포함된 기업명 ticker 가져오기

In [3]:
import FinanceDataReader as fdr 

# # 국내 주식
# kospi_df = fdr.StockListing('KOSPI')
# kosdaq_df = fdr.StockListing('KOSDAQ')

# # 미국주식
# nasdaq_df = fdr.StockListing('NASDAQ')
# nyse_df = fdr.StockListing('NYSE')

In [4]:
def get_korname(ticker):
    url = f"https://m.stock.naver.com/worldstock/stock/{ticker}/total" # 수정

    html_content = requests.get(url).content
    soup = BeautifulSoup(html_content, 'html.parser')
    try:
        korname = soup.find('meta', {'property': 'og:title'})['content'].replace(" - 네이버페이 증권", "")
        return korname
    except:
        print(f"{ticker} Korean Name Not Found")
        return None

In [5]:
# nasdaq_df['KorName'] = ''

# for index, row in nasdaq_df.iterrows():
#     nasdaq_df.at[index, 'KorName'] = get_korname(row['Symbol'])

# nyse_df['KorName'] = ''

# for index, row in nyse_df.iterrows():
#     nyse_df.at[index, 'KorName'] = get_korname(row['Symbol'])

In [6]:
# nasdaq_df['Market'] = 'NASDAQ'
# nyse_df['Market'] = 'NYSE'

# columns = ['KorName', 'Symbol', 'Market']
# nasdaq_df = nasdaq_df[columns]
# nyse_df = nyse_df[columns]

In [7]:
# columns = ['Name', 'Code', 'Market']
# kospi_df = kospi_df[columns]
# kosdaq_df = kosdaq_df[columns]

# kospi_df.loc[:, 'Code'] = kospi_df['Code'] + ".KS"
# kosdaq_df.loc[:, 'Code'] = kosdaq_df['Code'] + ".KQ"

In [8]:
# columns = ['Name', 'Ticker', 'Market']

# kospi_df.columns = columns
# kosdaq_df.columns = columns
# nasdaq_df.columns = columns
# nyse_df.columns = columns

In [9]:
# company_df = pd.concat([kospi_df, kosdaq_df, nasdaq_df, nyse_df])
# company_df.to_excel("company_listing.xlsx", index=0)

In [10]:
df = pd.read_excel("company_listing.xlsx")
df = df[df['Name'] != '네이버페이 증권'] # 이후 보완하겠음
df.to_excel("company_listing.xlsx", index=0)

#### 뉴스 제목

In [12]:
from langchain.agents import tool
from pydantic import BaseModel, Field

In [13]:
# api key 불러오기
naver_client_id = os.getenv('NAVER_CLIENT_ID')
naver_client_secret = os.getenv('NAVER_CLIENT_SECRET')

In [14]:
@tool
def get_navernews(query: str, api_key=naver_client_id, api_secret_key=naver_client_secret) -> dict:
    """
    네이버 API를 사용해 특정 쿼리에 대한 네이버 블로그 검색 결과를 반환하는 함수입니다.
    """

    encText = urllib.parse.quote(query)
    url = "https://openapi.naver.com/v1/search/news?query=" + encText # JSON 결과

    header={
        "X-Naver-Client-Id":api_key,
        "X-Naver-Client-Secret":api_secret_key
    }

    payload={
        "query":bytes(query, 'UTF-8'),
        "display":10, # 100
        "sort":'sim' # sim(정확도순)/date(날짜순)
    }

    response = requests.get(url,params=payload, headers=header)
    data = json.loads(response.text)
    return data['items']

### 주가 정보

In [19]:
import yfinance as yf
from langchain.agents import tool
from datetime import datetime, timedelta


#가장 최신일 주가 정보


def get_today_stockprice(ticker: str) -> dict:
    stock = yf.Ticker(ticker)

    sp_df = stock.history(
        period="1d"
        # interval='1d', # 종목에 따라 1m도 가능
        # # start='2022-01-01',
        # actions=True,
        # auto_adjust=True
    )
    return sp_df.to_dict(orient='index')

@tool
def get_structured_stockprice(ticker: str) -> dict:
    """기업의 종목코드를 넣으면 해당 기업의 가장 최신 주가 정보(일단위)를 반환합니다."""
    for inner_dict in get_today_stockprice(ticker).values():
        open_value = inner_dict['Open']
        hign_value = inner_dict['High']
        low_value = inner_dict['Low']
        close_value = inner_dict['Close']
        volume_value = inner_dict['Volume']
        break

    today_sp = {
        '시가':open_value,
        '종가':close_value,
        '고가':hign_value,
        '저가':low_value,
        '거래량':volume_value
    }
    return today_sp


In [22]:
# 전달 대비 변화
def get_dailystockprice(ticker: str) -> dict:
    one_month_ago = datetime.now() - timedelta(days=30)
    formatted_date = one_month_ago.strftime('%Y-%m-%d')
    
    stock = yf.Ticker(ticker)

    sp_df = stock.history(
        interval='1d', # 종목에 따라 1m도 가능
        start=formatted_date,
        actions=True,
        auto_adjust=True
    )
    return sp_df.to_dict(orient='index')

In [23]:
@tool
def get_structured_dprice(ticker: str) -> dict:
    """전날과 비교한 주가변화율을 반환합니다."""
    data = get_dailystockprice(ticker)
    first_close = list(data.values())[-2]['Close']
    last_close = list(data.values())[-1]['Close']

    # 변화율 계산
    change_rate = round(((last_close - first_close) / first_close) * 100, 2)

    return change_rate

In [24]:
@tool
def get_structured_mprice(ticker: str) -> dict:
    """한달전과 비교한 주가변화율을 반환합니다."""
    data = get_dailystockprice(ticker)
    first_close = list(data.values())[0]['Close']
    last_close = list(data.values())[-1]['Close']

    # 변화율 계산
    change_rate = round(((last_close - first_close) / first_close) * 100, 3)

    return change_rate

In [25]:
# 전년도 대비 변화
def get_ystockprice(ticker: str) -> dict:
    one_year_ago = datetime.now() - timedelta(days=365)
    formatted_date = one_year_ago.strftime('%Y-%m-%d')
    
    stock = yf.Ticker(ticker)

    sp_df = stock.history(
        interval='1d', # 종목에 따라 1m도 가능
        start=formatted_date,
        actions=True,
        auto_adjust=True
    )
    return sp_df.to_dict(orient='index')

@tool
def get_structured_yprice(ticker: str) -> dict:
    """작년과 비교한 주가변화율을 반환합니다."""
    data = get_ystockprice(ticker)
    first_close = list(data.values())[0]['Close']
    last_close = list(data.values())[-1]['Close']

    # 변화율 계산
    change_rate = round(((last_close - first_close) / first_close) * 100, 2)

    return change_rate

### 주요 재무 지표

In [26]:
import yfinance as yf

def get_financials(ticker):
    stock = yf.Ticker(ticker)
    financial_dict = stock.get_income_stmt(as_dict=True,freq="quarterly")

    return financial_dict

def calculate_change(data, column_name):
    sorted_dates = sorted(data.keys())

    latest_date = sorted_dates[-1]
    previous_date = sorted_dates[-2]

    latest_net_income = data[latest_date][column_name]
    previous_net_income = data[previous_date][column_name]

    change = latest_net_income - previous_net_income
    change_percentage = round((change / previous_net_income) * 100, 2)

    return latest_net_income, change_percentage

In [27]:
financial_data = get_financials('AAPL')
calculate_change(financial_data, 'NetIncome') # 저번 분기 대비 순이익 변화

(23636000000.0, -30.31)

### 구글 검색

In [28]:
google_api_key = os.getenv('GOOGLE_API_KEY')
google_searchengine_id = os.getenv('GOOGLE_SEARCHENGINE_ID')

In [29]:
query = "삼성전자"  # 검색할 쿼리
start_page = "1" # 몇 페이지를 검색할 것인지. 한 페이지 당 10개의 게시물을 받아들일 수 있습니다. 

url = f"https://www.googleapis.com/customsearch/v1?key={google_api_key}&cx={google_searchengine_id}&q={query}&start={start_page}"

response = requests.get(url).json()['items']
response

[{'kind': 'customsearch#result',
  'title': '삼성전자',
  'htmlTitle': '<b>삼성전자</b>',
  'link': 'https://www.samsung.com/sec/',
  'displayLink': 'www.samsung.com',
  'snippet': '자급제와 통신사폰 모두 삼성닷컴에서! 다양한 구매 혜택을 지금 확인해보세요. 더 알아보기 구매 혜택 보기.',
  'htmlSnippet': '자급제와 통신사폰 모두 <b>삼성</b>닷컴에서! 다양한 구매 혜택을 지금 확인해보세요. 더 알아보기 구매 혜택 보기.',
  'formattedUrl': 'https://www.samsung.com/sec/',
  'htmlFormattedUrl': 'https://www.samsung.com/sec/',
  'pagemap': {'cse_thumbnail': [{'src': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRK3NpT5Ij1YqSDFy0rpPB54FMIATJN4Odx9lg-dYkc42Z-dISSLYv2H18r&s',
     'width': '225',
     'height': '225'}],
   'metatags': [{'og:image': 'https://images.samsung.com/kdp/st/1/5ab39f9a-9f76-4be5-b7e2-ecf40795aade.png',
     'twitter:card': 'Summary',
     'twitter:title': 'Samsung 대한민국 | 모바일 | TV | 가전 | IT',
     'og:type': 'website',
     'og:site_name': 'Samsung sec',
     'twitter:url': 'https://www.samsung.com/sec/',
     'og:title': 'Samsung 대한민국 | 모바일 | TV | 가전 | IT',
 

In [30]:
def get_GoogleSearch(query: str, google_api_key: str, google_searchengine_id: str) -> dict:
    """입력한 쿼리에 대해 구글 검색 결과를 반환합니다."""

    lang = "lang_ko" # 언어
    num = 10 # 반환 검색결과수
    # sort = # 정렬

    url = f"https://www.googleapis.com/customsearch/v1?key={google_api_key}&cx={google_searchengine_id}&q={query}&lr={lang}&num={num}"

    response = requests.get(url).json()
    return response['items']

In [31]:
get_GoogleSearch('애플 경영진', google_api_key, google_searchengine_id)[1]

{'kind': 'customsearch#result',
 'title': '틀:Apple의 경영진 - 나무위키',
 'htmlTitle': '틀:<b>Apple</b>의 <b>경영진</b> - 나무위키',
 'link': 'https://namu.wiki/w/%ED%8B%80:Apple%EC%9D%98%20%EA%B2%BD%EC%98%81%EC%A7%84',
 'displayLink': 'namu.wiki',
 'snippet': 'May 20, 2024 ... 틀:Apple의 경영진 ; 루카 마에스트리. CFO. 2013년 ; 케이트 애덤스. 총괄고문변호사. 2017년 ; 디어드레 오브라이언. 수석부사장 (소매). 1988년 ; 제프\xa0...',
 'htmlSnippet': 'May 20, 2024 <b>...</b> 틀:<b>Apple</b>의 <b>경영진</b> ; 루카 마에스트리. CFO. 2013년 ; 케이트 애덤스. 총괄고문변호사. 2017년 ; 디어드레 오브라이언. 수석부사장 (소매). 1988년 ; 제프&nbsp;...',
 'formattedUrl': 'https://namu.wiki/w/틀:Apple의%20경영진',
 'htmlFormattedUrl': 'https://namu.wiki/w/틀:Apple의%20<b>경영진</b>',
 'pagemap': {'cse_thumbnail': [{'src': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTw5NOkJDMhOOoFJE9dOfUBycCs10ZVWaBdFvsNjLJyGsSUnvEX-F55xro&s',
    'width': '225',
    'height': '225'}],
  'metatags': [{'application-name': '나무위키',
    'msapplication-starturl': '/w/%EB%82%98%EB%AC%B4%EC%9C%84%ED%82%A4:%EB%8C%80%EB%AC%B8',
    'og:i

### 주요 경제 지표

In [32]:
bank_api_id = os.getenv('BANK_API_ID')
# 아직 안 만들었음 (이정도는 llm이 알지 않을까 싶어서..)

### TEST

In [33]:
from custom_model import hyperclovax
from modules import StockTickerFinder, ConversationManager

In [34]:
finder = StockTickerFinder()

query = "애플 살까?"

file_path = "/company_listing.xlsx"  
finder.load_company_data(file_path)
company_name, ticker = finder.find_most_similar_company(query)
print(f"The most similar company is: {company_name} with ticker: {ticker}")

# 뉴스
news = get_navernews(company_name)
news_list = []

for article in news:
    # print(article)
    clean_title = re.sub('<[^<]+?>', '', article['title'])
    clean_title = clean_title.replace('&quot;', '"')
    news_list.append(clean_title)

# 주가 정보
sp = get_structured_stockprice(ticker)
dsp = get_structured_dprice(ticker)
ysp = get_structured_yprice(ticker)

# 재무 정보 Net Income만
financial_data = get_financials(ticker)
calculate_change(financial_data, 'NetIncome') # 저번 분기 대비 순이익 변화

# llm 생성
hyperclovax = hyperclovax(
    host='https://clovastudio.stream.ntruss.com',
    api_key='NTA0MjU2MWZlZTcxNDJiYxrm80PYC0F2nPTOcfcl6q+St7u/ykmVS25U0nuONAIC',
    api_key_primary_val='fb8TL0kwYo5Mz8XWy0jwNuIpHmEiAqIjxkesYjnU',
    request_id='8507dde5-9fff-45d6-ac49-3a8364f69291'
)

The most similar company is: 애플 with ticker: AAPL


In [36]:
conversation_manager = ConversationManager()
conversation_manager.add_message("system", """너는 투자 전문가야. 너는 아래 요인을 고려하여 투자 점수를 계산하고 이에 따라 투자 여부를 결정해. 반드시 지정한 최종형식에 따라 결과를 출력해\r\n\r\n
        - 거시 경제 상황\n
        - 기업 경쟁력(매출 변화, 시장점유율)\n
        - 최근 뉴스: {news_list}\n
        - 주가: {sp}, 전날 대비 {dsp}, 전년도 대비 {ysp}\n
        \n
        
        <투자 점수 계산식>
        투자 점수 = 0.3 * 거시 경제 요인 + 0.2 * 기업경쟁력 + 0.3 * 최근 뉴스 + 0.2 * 주가 \n
        최종 투자 점수와 그렇게 판단한 근거를 반드시 아래 형식에 따라 구체적으로 설명해줘\n
        
        <최종 형식>\n
        투자 여부: [긍정/부정]\n
        투자 점수: [최종 점수]\n
        구체적인 근거: [근거 설명]\n
""")

In [37]:
conversation_manager.add_message("user", "현재 애플에 투자를 할까?")
prompt = conversation_manager.get_payload()
response = hyperclovax._call(prompt=prompt)


response_content = response['content'] 
print(response_content)
conversation_manager.add_message("assistant", response) # 답변도 저장

네, 현재 애플에 대한 투자 여부를 결정하기 위해 다음과 같은 요소들을 고려할 수 있습니다.  

- 거시 경제 상황: 현재 글로벌 경제는 코로나19로 인해 불확실성이 높은 상황입니다. 그러나 애플은 세계 최대의 스마트폰 및 태블릿 제조업체로서 안정적인 매출을 유지하고 있으며, 이를 바탕으로 경기 회복 시 빠른 성장이 기대됩니다. 

- 기업 경쟁력: 애플은 매년 혁신적인 제품을 출시하며, 전 세계적으로 높은 브랜드


In [38]:
conversation_manager.get_payload()

{'messages': [{'role': 'system',
   'content': '\n            너는 투자 전문가야. 너는 아래 요인을 고려하여 투자 점수를 계산하고 이에 따라 투자 여부를 결정해. 반드시 지정한 최종형식에 따라 결과를 출력해\r\n\r\n\n            - 거시 경제 상황\r\n\n            - 기업 경쟁력(매출 변화, 시장점유율)\r\n\n            - 최근 뉴스: {news_list}\r\n\n            - 주가: {sp}, 전날 대비 {dsp}, 전년도 대비 {ysp}\r\n\n            \r\n\r\n\n            \n            <투자 점수 계산식>\n            투자 점수 = 0.3 * 거시 경제 요인 + 0.2 * 기업경쟁력 + 0.3 * 최근 뉴스 + 0.2 * 주가 \r\n\r\n\n            최종 투자 점수와 그렇게 판단한 근거를 구체적으로 설명해줘\n\n\n            \n            <최종 형식>\n#####\n\n            투자 여부: [긍정/부정]\n\n            투자 점수: [최종 점수]\n\n            구체적인 근거: [근거 설명]\n\n            #####\n'},
  {'role': 'user', 'content': '현재 애플에 투자를 할까?'},
  {'role': 'assistant',
   'content': {'role': 'assistant',
    'content': '네, 현재 애플에 대한 투자 여부를 결정하기 위해 다음과 같은 요소들을 고려할 수 있습니다.  \n\n- 거시 경제 상황: 현재 글로벌 경제는 코로나19로 인해 불확실성이 높은 상황입니다. 그러나 애플은 세계 최대의 스마트폰 및 태블릿 제조업체로서 안정적인 매출을 유지하고 있으며, 이를 바탕으로 경기 회복 시 빠른 성장이 기대됩니다. \n\n- 기업 경쟁력: 애플은 매